In [3]:
include("../Inversion/Plot.jl")
include("../Inversion/KalmanInversion.jl")
include("Linear.jl")

forward_aug (generic function with 1 method)

# Two Parameter Linear Inverse Problems: Optimization Approaches

Consider the 2-parameter linear inverse problem
$$y = G\theta + \eta.$$
with $\eta \sim \mathcal{N}(0, 0.1^2I)$. We explore the following three scenarios

#### non-singular (well-determined) system (NS) 
   $$
   y = \begin{bmatrix}
        3 \\
        7 
       \end{bmatrix} \quad
       G = \begin{bmatrix}
        1 & 2\\
        3 & 4
    \end{bmatrix} \quad
    \theta_{ref} = \begin{bmatrix}
        1 \\
        1 
       \end{bmatrix};
   $$ 
   
#### over-determined system (OD)
$$ y = \begin{bmatrix}
        3 \\
        7 \\
        10
       \end{bmatrix} \quad
       G = \begin{bmatrix}
        1 & 2\\
        3 & 4 \\
        5 & 6
    \end{bmatrix} \quad
    \theta_{ref} = \begin{bmatrix}
        1/3 \\
        17/12
       \end{bmatrix};
$$ 
   
#### under-determined system (UD)
$$
y = \begin{bmatrix}
        3
       \end{bmatrix} \quad
       G = \begin{bmatrix}
        1 & 2\\
    \end{bmatrix} \quad
       \theta_{ref} = \begin{bmatrix}
        1 \\
        1 
       \end{bmatrix}
       + c\begin{bmatrix}
        2 \\
        -1 
       \end{bmatrix}, \,\,c \in \mathcal{R}.
$$
   
We define $$\displaystyle \theta_{ref} = argmin_{\theta}\frac{1}{2}\lVert\Sigma_{\eta}^{-\frac{1}{2}}(y - G\theta)\rVert^2,$$ and 
note that, in the UD case, $\theta_{ref}$ comprises a one-parameter family of possible solutions. We also note that $y = G\theta_{ref}$ for NS; and 
$y = G\theta_{ref}(c^\dagger)$ for UD, with $c^{\dagger}=0$; but for OD $y \ne  G\theta_{ref}.$ 

In [4]:
function Two_Param_Linear_Test(problem_type::String, θ0_bar, θθ0_cov)
    
    N_θ = length(θ0_bar)

    
    if problem_type == "under-determined"
        # under-determined case
        θ_ref = [0.6, 1.2]
        G = [1.0 2.0;]
        
        y = [3.0;]
        Σ_η = Array(Diagonal(fill(0.1^2, size(y))))
        
        
    
    elseif problem_type == "over-determined"
        # over-determined case
        θ_ref = [1/3, 8.5/6]
        G = [1.0 2.0; 3.0 4.0; 5.0 6.0]
        
        y = [3.0;7.0;10.0]
        Σ_η = Array(Diagonal(fill(0.1^2, size(y))))
        
        
    else
        error("Problem type : ", problem_type, " has not implemented!")
    end
    
    Σ_post = inv(G'*(Σ_η\G) + inv(θθ0_cov))
    θ_post = θ0_bar + Σ_post*(G'*(Σ_η\(y - G*θ0_bar)))
    

    return θ_post, Σ_post, G, y, Σ_η
end


function construct_cov(x::Array{FT,2}) where {FT<:AbstractFloat}
    
    x_mean = dropdims(mean(x, dims=1), dims=1)
    N_ens, N_x = size(x)
    
    
    x_cov = zeros(FT, N_x, N_x)
    
    for i = 1: N_ens
        x_cov .+= (x[i,:] - x_mean)*(x[i,:] - x_mean)'
    end
    
    return x_cov/(N_ens - 1)
end

construct_cov (generic function with 9 methods)

## Unscented Kalman Inversion

When the unscented Kalman inversion is applied with $\theta_0 \sim \mathcal{N}(0, I)$, 
the convergence of the $\theta$ for all three scenarios are depicted, in terms of the $L_2$ error $\lVert m_n - \theta_{ref}\rVert_2$ 
Exponential convergence is achieved!

In [5]:
FT = Float64
uki_objs = Dict()
mean_errors = Dict()

Random.seed!(123)
α_reg = 1.0
update_freq = 1
N_iter = 30
N_θ = 2
θ0_mean = zeros(FT, N_θ)
θθ0_cov = Array(Diagonal(fill(1.0^2, N_θ)))
θθ0_cov_sqrt = θθ0_cov
for problem_type in ("under-determined", "over-determined")
    
    θ_post, Σ_post, G, y, Σ_η = Two_Param_Linear_Test(problem_type, θ0_mean, θθ0_cov)
    
    N_y = length(y)
    
    s_param = Setup_Param(G, N_θ, N_y)
    s_param_aug = Setup_Param(G, N_θ, N_y+N_θ)
    
    
    
    y_aug = [y ;θ0_mean]
    Σ_η_aug = [Σ_η zeros(Float64, N_y, N_θ); zeros(Float64, N_θ, N_y)  θθ0_cov]
 
    # UKI
    uki_obj = UKI_Run(s_param_aug, forward_aug, 
    θ0_mean, θθ0_cov,
    y_aug, Σ_η_aug,
    α_reg,
    update_freq,
    N_iter)
    
    
    N_θ_s = 5
    N_θ_l = 100
    # EKI
    @info "To use the original EKI, uncomment EKI.jl line 88-89"
    filter_type = "EKI"
    N_ens = N_θ_s
    eki_obj_s = EKI_Run(s_param, forward, 
    filter_type,
    θ0_mean, θθ0_cov_sqrt,
    N_ens,
    y, Σ_η,
    1.0,
    0,
    N_iter)
    
    N_ens = N_θ_l
    eki_obj_l = EKI_Run(s_param, forward, 
    filter_type,
    θ0_mean, θθ0_cov_sqrt,
    N_ens,
    y, Σ_η,
    1.0,
    0,
    N_iter)
    
    
    uki_errors = zeros(FT, N_iter+1, 2)
    eki_s_errors = zeros(FT, N_iter+1, 2)
    eki_l_errors = zeros(FT, N_iter+1, 2)
    
    for i = 1:N_iter+1
        uki_errors[i, 1] = norm(uki_obj.θ_mean[i] .- θ_post)/norm(θ_post)
        uki_errors[i, 2] = norm(uki_obj.θθ_cov[i] .- Σ_post)/norm(Σ_post)
        
        eki_s_errors[i, 1] = norm(dropdims(mean(eki_obj_s.θ[i], dims=1), dims=1) .- θ_post)/norm(θ_post)
        eki_s_errors[i, 2] = norm(construct_cov(eki_obj_s.θ[i]) .- Σ_post)/norm(Σ_post)
        
        eki_l_errors[i, 1] = norm(dropdims(mean(eki_obj_l.θ[i], dims=1), dims=1) .- θ_post)/norm(θ_post)
        eki_l_errors[i, 2] = norm(construct_cov(eki_obj_l.θ[i]) .- Σ_post)/norm(Σ_post)
        
    end
    
    ites = Array(0:N_iter)
    
    fig, ax = PyPlot.subplots(nrows = 1, ncols=2, sharex=false, sharey=false, figsize=(12,6))
    ax[1].semilogy(ites, uki_errors[:, 1],   "--o", color = "C0", fillstyle="none", label="UKI")
    ax[1].semilogy(ites, eki_s_errors[:, 1], "--*", color = "C1", fillstyle="none", label="EKI (J=$(N_θ_s))")
    ax[1].semilogy(ites, eki_l_errors[:, 1], "--s", color = "C2", fillstyle="none", label="EKI (J=$(N_θ_l))")
    ax[1].set_xlabel("Iterations")
    ax[1].set_ylabel("Rel. mean error")
    ax[1].grid("on")
    ax[1].legend()
    
    ax[2].semilogy(ites, uki_errors[:, 2],   "--o", color = "C0", fillstyle="none", label="UKI")
    ax[2].semilogy(ites, eki_s_errors[:, 2], "--*", color = "C1", fillstyle="none", label="EKI (J=$(N_θ_s))")
    ax[2].semilogy(ites, eki_l_errors[:, 2], "--s", color = "C2", fillstyle="none", label="EKI (J=$(N_θ_l))")
    ax[2].set_xlabel("Iterations")
    ax[2].set_ylabel("Rel. covariance error")
    ax[2].grid("on")
    ax[2].legend()
    fig.tight_layout()
    fig.savefig("Linear-$(problem_type).pdf")
end



┌ Info: To use the original EKI, uncomment EKI.jl line 88-89
└ @ Main In[5]:39


LoadError: MethodError: no method matching EKIObj(::String, ::Vector{String}, ::Int64, ::Vector{Float64}, ::Matrix{Float64}, ::Vector{Float64}, ::Matrix{Float64}, ::Float64, ::Int64)
[0mClosest candidates are:
[0m  EKIObj(::String, ::Vector{String}, ::IT, ::Array{FT}, ::Matrix{FT}, ::Vector{FT}, ::Any, ::FT) where {FT<:AbstractFloat, IT<:Int64} at ~/Desktop/Code/InverseProblems-master.jl/Inversion/EKI.jl:55
[0m  EKIObj(::String, ::Vector{String}, ::IT, ::Array{FT}, ::Matrix{FT}, ::Vector{FT}, ::Any, ::FT, [91m::FT[39m) where {FT<:AbstractFloat, IT<:Int64} at ~/Desktop/Code/InverseProblems-master.jl/Inversion/EKI.jl:55
[0m  EKIObj(::String, ::Vector{String}, [91m::Array{Matrix{FT}, 1}[39m, [91m::Array{Vector{FT}, 1}[39m, [91m::Vector{FT}[39m, ::Any, [91m::IT[39m, [91m::IT[39m, ::IT, [91m::FT[39m, [91m::FT[39m, [91m::IT[39m) where {FT<:AbstractFloat, IT<:Int64} at ~/Desktop/Code/InverseProblems-master.jl/Inversion/EKI.jl:28

In [4]:
problem_type  = "over-determined"

    θ_post, Σ_post, G, y, Σ_η = Two_Param_Linear_Test(problem_type, θ0_mean, θθ0_cov)
    
    N_y = length(y)
    
    s_param = Setup_Param(G, N_θ, N_y)
    s_param_aug = Setup_Param(G, N_θ, N_y+N_θ)
    
    
    
    y_aug = [y ;θ0_mean]
    Σ_η_aug = [Σ_η zeros(Float64, N_y, N_θ); zeros(Float64, N_θ, N_y)  θθ0_cov]
 
    # UKI
    uki_obj = UKI_Run(s_param_aug, forward_aug, 
    θ0_mean, θθ0_cov,
    y_aug, Σ_η_aug,
    α_reg,
    update_freq,
    N_iter)
    
    
    N_θ_s = 5
    N_θ_l = 100
    # EKI
    @info "To use the original EKI, uncomment EKI.jl line 88-89"
    filter_type = "EKI"
    N_ens = N_θ_s
    eki_obj_s = EKI_Run(s_param, forward, 
    filter_type,
    θ0_mean, θθ0_cov_sqrt,
    N_ens,
    y, Σ_η,
    1.0,
    0,
    N_iter)
    
    N_ens = N_θ_l
    eki_obj_l = EKI_Run(s_param, forward, 
    filter_type,
    θ0_mean, θθ0_cov_sqrt,
    N_ens,
    y, Σ_η,
    1.0,
    0,
    N_iter)
    
    # EKS
    N_ens = N_θ_s
    eks_obj_s = EKS_Run(s_param, forward, 
    θ0_mean, θθ0_cov,
    N_ens,
    y, Σ_η,
    N_iter)
    
    @info "EKS small J t = ", sum(eks_obj_s.Δt)
    
    N_ens = N_θ_l
    eks_obj_l = EKS_Run(s_param, forward, 
    θ0_mean, θθ0_cov,
    N_ens,
    y, Σ_η,
    N_iter)
    
    @info "EKS large J t = ", sum(eks_obj_s.Δt)
    
    uki_errors = zeros(FT, N_iter+1, 2)
    eki_s_errors = zeros(FT, N_iter+1, 2)
    eki_l_errors = zeros(FT, N_iter+1, 2)
    eks_s_errors = zeros(FT, N_iter+1, 2)
    eks_l_errors = zeros(FT, N_iter+1, 2)
    
    for i = 1:N_iter+1
        uki_errors[i, 1] = norm(uki_obj.θ_mean[i] .- θ_post)/norm(θ_post)
        uki_errors[i, 2] = norm(uki_obj.θθ_cov[i] .- Σ_post)/norm(Σ_post)
        
        eki_s_errors[i, 1] = norm(dropdims(mean(eki_obj_s.θ[i], dims=1), dims=1) .- θ_post)/norm(θ_post)
        eki_s_errors[i, 2] = norm(construct_cov(eki_obj_s.θ[i]) .- Σ_post)/norm(Σ_post)
        
        eki_l_errors[i, 1] = norm(dropdims(mean(eki_obj_l.θ[i], dims=1), dims=1) .- θ_post)/norm(θ_post)
        eki_l_errors[i, 2] = norm(construct_cov(eki_obj_l.θ[i]) .- Σ_post)/norm(Σ_post)
        
        eks_s_errors[i, 1] = norm(dropdims(mean(eks_obj_s.θ[i], dims=1), dims=1) .- θ_post)/norm(θ_post)
        eks_s_errors[i, 2] = norm(construct_cov(eks_obj_s.θ[i]) .- Σ_post)/norm(Σ_post)
        
        eks_l_errors[i, 1] = norm(dropdims(mean(eks_obj_l.θ[i], dims=1), dims=1) .- θ_post)/norm(θ_post)
        eks_l_errors[i, 2] = norm(construct_cov(eks_obj_l.θ[i]) .- Σ_post)/norm(Σ_post)
        
    end

┌ Info: To use the original EKI, uncomment EKI.jl line 88-89
└ @ Main In[4]:27
┌ Info: ("EKS small J t = ", 20.579847851940084)
└ @ Main In[4]:57
┌ Info: ("EKS large J t = ", 20.579847851940084)
└ @ Main In[4]:66


In [6]:
eki_obj_s.θ[end]

5×2 Matrix{Float64}:
 0.371736  1.38625
 0.333511  1.41589
 0.388344  1.37462
 0.360575  1.3932
 0.379444  1.38222

In [7]:
eki_obj_l.θ[end]

100×2 Matrix{Float64}:
 0.362574  1.39662
 0.33597   1.41495
 0.294256  1.44945
 0.350994  1.40544
 0.333933  1.42108
 0.367327  1.38883
 0.298989  1.44142
 0.292762  1.44976
 0.342192  1.40987
 0.336544  1.41331
 0.37291   1.38452
 0.321566  1.42602
 0.333312  1.4151
 ⋮         
 0.317159  1.42844
 0.331934  1.41939
 0.369265  1.38717
 0.353708  1.4038
 0.354847  1.40128
 0.349713  1.40106
 0.333509  1.41886
 0.317831  1.42711
 0.345463  1.40727
 0.393244  1.36677
 0.321012  1.42464
 0.286846  1.4491